In [1]:
import numpy as np
import pandas as pd

dtypes_raw = {
    'elapsed_time': np.int32,
    'event_name': 'category',
    'name': 'category',
    'level': np.uint8,
    'room_coor_x': np.float32,
    "index": np.int32,
    'room_coor_y': np.float32,
    'screen_coor_x': np.float32,
    'screen_coor_y': np.float32,
    'hover_duration': np.float32,
    'text': 'category',
    'fqid': 'category',
    'room_fqid': 'category',
    'text_fqid': 'category',
    'fullscreen': 'category',
    'hq': 'category',
    'music': 'category',
    'level_group': 'category'}

dtypes_pp = {
            'level': np.uint8,  
            'session_id': np.int64,
            'level_group': 'category',
            'event_name': np.uint8,
            'name': np.uint8,
            'fqid': np.uint8,
            'room_fqid': np.uint8,
            'text_fqid': np.uint8,
            'fullscreen': np.uint8,
            'hq': np.uint8,
            'music': np.uint8,
            'hover_duration_mean': np.float32,
            'difference_clicks_mean': np.float32,
            "distance_clicks_mean": np.float32,
            "screen_distance_clicks_mean": np.float32,            
            'elapsed_time_std': np.float32,
            'page_std': np.float32,
            'room_coor_x_std': np.float32,
            'room_coor_y_std': np.float32,
            'screen_coor_x_std': np.float32,
            'screen_coor_y_std': np.float32,
            'hover_duration_std': np.float32,
            'difference_clicks_std': np.float32,
            "distance_clicks_std": np.float32,
            "screen_distance_clicks_std": np.float32,
            'index_sum_of_actions': np.int32,
            'difference_clicks_max': np.float32,
            'elapsed_time_max': np.float32,
            'clicks_per_second': np.float32,
            "sum_distance_clicks_max": np.float32}

# 1. Load your model: [your model name here] (in this case: steves optuna xgboost decision tree)

In [2]:
import xgboost as xgb #delete?
import pickle

# Load the model from file
with open('/kaggle/input/best-models-dict-optunahp/best_models_dict_optunahp.pkl', 'rb') as f:
    models = pickle.load(f)

# 2. Run the model
DON'T READ THIS: This jo_wilder-kind-of-timeseries-API simulates the real world scenario for the model you've trained. The iter_test gives you pairs of: 
1. X_test_raw: gamelog data of a single session_id from only one level_group 
2. y_test_label_preds: and secondly a dataframe (similiar to raw/train_labels.csv) in which you have to fill in your predictions for each question of that level_group.

**Additional WARNINGS**:
- submission.csv is NOT automatically reseted, even after a Kernel reboot the iter_test will continue to add sample submissions to the submission.csv. Therefor, before your final run, do a factory reset (Run > Factory reset).
- the iter_tester needs a kernel reboot after each run - or you will get the error: "Exception: You can only iterate over `iter_test()` once."

In [3]:
# Unfortunately this doesn't work because you don't have the rights to do this on Kaggle:
# os.remove("/kaggle/working/submission.csv")

# Import utils for feature engineering
import sys
sys.path.insert(1, '/kaggle/input/utils-for-preprocessing')
import data_preprocessing_steve

# Import the time series API
import jo_wilder
env = jo_wilder.make_env()
iter_test = env.iter_test()

limits = {'0-4':(1,4), '5-12':(4,14), '13-22':(14,19)}

for (X_test_raw, y_test_label_preds) in iter_test: # Raw data from only ONE session_id and level_group in each loop 
    # Pre-process the raw test data: the pipeline returns a dict with datasets split by level_group.
    X_test_dict = data_preprocessing_steve.pp_pipeline_noah(data=X_test_raw, file_path=None, flatten=True, saveIntermediateFiles=False,
                     dtypes=dtypes_raw, output=True)
    
    # Since we only have data from one level_group, we just take the first dataset out of the dict.
    grp = list(X_test_dict.keys())[0]
    X_test = X_test_dict[grp].drop(['session_id_1', 'level_group_1'], axis=1)
    
    # Here we adapt the dtypes of our processed data
    for column in X_test.columns:
        base_name = column.rsplit('_', 1)[0]  # get the base name by splitting on the last "_" character
        if base_name in dtypes_pp:
            column_number = column.rsplit('_', 1)[1]  # get the number from the index by splitting on the last "_" character
            new_column_name = f"{base_name}_{column_number}"  # construct the new column name
            column_dtype = dtypes_pp[base_name]
            X_test[new_column_name] = X_test[column].astype(column_dtype)  # set the same data type for all columns with the same base name

    
    a,b = limits[grp]
    
    # We loop for each question - t
    for t in range(a,b):
        # Get the model for question t
        model = models[f'{t}']
        # Make prediction
        y_preds = model.predict(X_test)
        # Write prediction to y_test_label_preds
        mask = y_test_label_preds.session_id.str.contains(f'q{t}')
        y_test_label_preds.loc[mask,'correct'] = y_preds[0]
    
    # Add your predictions of this session_id to the submission before getting the next session_id data
    env.predict(y_test_label_preds)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


# 3. Quick submission EDA
- Check if your submission file looks correct

In [4]:
df = pd.read_csv('submission.csv')
print(df.shape)
print(df.head(20))
print(df.correct.mean())

(54, 2)
               session_id  correct
0    20090109393214576_q1        1
1    20090109393214576_q2        1
2    20090109393214576_q3        1
3    20090109393214576_q4        1
4    20090109393214576_q5        1
5    20090109393214576_q6        1
6    20090109393214576_q7        1
7    20090109393214576_q8        1
8    20090109393214576_q9        1
9   20090109393214576_q10        0
10  20090109393214576_q11        1
11  20090109393214576_q12        1
12  20090109393214576_q13        1
13  20090109393214576_q14        1
14  20090109393214576_q15        1
15  20090109393214576_q16        1
16  20090109393214576_q17        1
17  20090109393214576_q18        1
18   20090312143683264_q1        1
19   20090312143683264_q2        1
0.7962962962962963


In [5]:
## the end result is a submission file containing all test session predictions
! head submission.csv

session_id,correct
20090109393214576_q1,1
20090109393214576_q2,1
20090109393214576_q3,1
20090109393214576_q4,1
20090109393214576_q5,1
20090109393214576_q6,1
20090109393214576_q7,1
20090109393214576_q8,1
20090109393214576_q9,1


# 4. Final steps for submission
* In the top right corner choose "Save Version" with Version type "Save & Run All (Commit)". Note the correct Version for later.
* In the lower left corner you'll see the process running. When it's done, press on the "Open in viewer" option and check if the output is a submission.csv with 1.29 kB
* Then go to the competition page, choose "Submit Predictions", choose your file with the correct version and correct output file - submit and you're done.
* Your submitted notebook will now be re-runned with private test data by the competition host. The public socre will appear in your submission section.
* The scoring will take a while...